In [4]:
import pandas as pd

# Load data
df = pd.read_csv(r'C:\Users\Akash\Desktop\electricity2\house_1_daily.csv', parse_dates=['timestamp'])

# Feature extraction
df['hour'] = df['timestamp'].dt.hour
df['day'] = df['timestamp'].dt.day
agg = df.groupby(['appliance', 'day']).agg({
    'power_kwh': ['mean', 'std', 'max', 'min']
}).reset_index()

agg.columns = ['appliance', 'day', 'mean_power', 'std_power', 'max_power', 'min_power']


In [5]:
agg.head()

,appliance,day,mean_power,std_power,max_power,min_power
0,coffee_machine,1,0.024957,0.033467,0.121980,0.0
1,coffee_machine,2,0.029479,0.060711,0.338794,0.0
2,coffee_machine,3,0.018576,0.024438,0.098392,0.0
3,coffee_machine,4,0.025735,0.068088,0.454872,0.0
4,coffee_machine,5,0.014978,0.018944,0.107054,0.0


In [6]:
from sklearn.ensemble import IsolationForest

# Train model
model = IsolationForest(contamination=0.05, random_state=42)
agg['anomaly'] = model.fit_predict(agg[['mean_power', 'std_power', 'max_power', 'min_power']])

# -1 = anomaly, 1 = normal
agg['status'] = agg['anomaly'].map({-1: 'Potential Fault', 1: 'Normal'})


In [7]:
import streamlit as st
import plotly.express as px

st.title("Appliance Health Monitoring Dashboard")

fig = px.bar(agg, x='day', y='mean_power', color='status', barmode='group',
             facet_col='appliance', title="Daily Appliance Power & Health Status")
st.plotly_chart(fig)

faulty = agg[agg['status'] == 'Potential Fault']
if not faulty.empty:
    st.warning("⚠️ Faulty Appliances Detected:")
    st.write(faulty[['appliance', 'day', 'mean_power', 'status']])


2025-04-18 17:08:58.344 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-18 17:08:59.202 
  command:

    streamlit run C:\Users\Akash\AppData\Roaming\Python\Python311\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-04-18 17:08:59.203 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-18 17:08:59.955 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-18 17:08:59.956 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-18 17:08:59.958 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-18 17:08:59.959 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-18 17:08:59.961 Thre